<a href="https://colab.research.google.com/github/alonba/eeg-bss/blob/main/FastICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne
!pip install pymatreader

In [19]:
# Fetch the data from the .set file
import numpy as np
import mne as mne
eeglab_epochs = mne.io.read_epochs_eeglab('./35_electrodes_filtered_epoched.set')
total_num_of_epochs = len(eeglab_epochs)
train_set_size = int(total_num_of_epochs*0.8)
test_set_size = total_num_of_epochs - train_set_size

Extracting parameters from c:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\eeg-bss\35_electrodes_filtered_epoched.set...
Not setting metadata
474 matching events found
No baseline correction applied
0 projection items activated
Ready.


In [47]:
train_set_indexes = []
test_set_indexes = []
for i in range(total_num_of_epochs):
    random_num = np.random.rand()
    if (random_num < 0.8):  # 80% chance of being here
        train_set_indexes.append(i)
    else:
        test_set_indexes.append(i)

380
94


In [49]:
train_set = eeglab_epochs[train_set_indexes]
test_set = eeglab_epochs[test_set_indexes]

In [52]:
mne.export.export_epochs('train_set.set',train_set)
mne.export.export_epochs('test_set.set',test_set)

In [6]:
ica = mne.preprocessing.ICA(n_components=35)  # Defaults to FastICA
ica.fit(eeglab_epochs)

Fitting ICA to data using 64 channels (please be patient, this may take a while)


<ipython-input-6-aaabcbea4706>:2: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(eeglab_epochs)


Selecting by number: 35 components
Fitting ICA took 32.9s.


Method,fastica
Fit,65 iterations on epochs (283800 samples)
ICA components,35
Explained variance,100.0 %
Available PCA components,64
Channel types,eeg
ICA components marked for exclusion,—


In [27]:
fitted_ica = ica.copy()
whitening = fitted_ica.pre_whitener_
mixing = fitted_ica.mixing_matrix_
unmixing = fitted_ica.unmixing_matrix_

In [28]:
ica.apply(eeglab_epochs)

Applying ICA to Epochs instance
    Transforming to ICA space (35 components)
    Zeroing out 0 ICA components
    Projecting back using 64 PCA components


Number of events,473
Events,S 1: 145S 2: 160S 4: 168
Time range,-0.200 – 0.998 sec
Baseline,off
